In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from random import sample 
from sklearn.cluster import KMeans
from collections import Counter
from itertools import combinations
import itertools
import math
import re

In [ ]:
# shortcut for selecting these three columns
LWH = ['L','W','H']

In [ ]:
def kmeans_loop(data,
                min_clusters,
                max_clusters,
                step):
    
    kmeans_kwargs = {
        "init": "random",
        "n_init": 10,
        "random_state": 1,
    }
    
    #create list to hold SSE values for each k
    labels = []
    i = 0
    for k in range(min_clusters, max_clusters+1, step):
        kmeans = KMeans(n_clusters = k, **kmeans_kwargs)
        kmeans.fit(data)
        labels.append(kmeans.labels_)
        i += 1

    return labels

In [ ]:
# read data file
os.chdir('C:\\Users\\85167\\OneDrive - Columbia Sportswear Company\\Desktop')
data_file = 'RVG Case Data.xlsx'
data_OG = pd.read_excel(data_file)
print('total records:', len(data_OG))

In [ ]:
# data file takes a long time to load, so make copy of data with only select columns
# (helpful during development if the data gets messed up; avoid reloading)
data = data_OG.copy()
data = data.loc[:,['Quantity','CaseVolume','CaseLength','CaseWidth','CaseHeight']]
data.columns = ['qty', 'V', 'L', 'W', 'H']
data.head()

In [ ]:
data_corr = data.corr()
data_corr = data_corr.style.background_gradient(cmap='coolwarm', vmin=-1, vmax=1)
data_corr

In [ ]:
len(data[data.V <= 6])/len(data)

In [ ]:
fig = px.histogram(data, x='V', range_x=[0,6])
fig.show()

In [ ]:
fig = px.histogram(data, x='L', range_x=[0,30])
fig.update_traces(xbins=dict( # bins used for histogram
        start=0,
        end=30,
        size=0.5
    ))
fig.show()

In [ ]:
fig = px.histogram(data, x='W', range_x=[0,30])
fig.update_traces(xbins=dict( # bins used for histogram
        start=0,
        end=30,
        size=0.5
    ))
fig.show()

In [ ]:
fig = px.histogram(data, x='H', range_x=[0,30])
fig.update_traces(xbins=dict( # bins used for histogram
        start=0,
        end=30,
        size=0.5
    ))
fig.show()

In [ ]:
# sample from records
def select_sample(data, nrecs):
    case_idx = sample(range(len(data)), nrecs)   
    sample_data = data.iloc[case_idx][LWH].reset_index(drop=True)
    return sample_data

# scale data (data in unit cube better for clustering)
def scale_down(data, data_max):
    data_scaled = data.copy()
    for i in range(len(data.columns)):
        col_i = data.columns[i]
        data_scaled[col_i] = data[col_i] / data_max[i]
        
    return data_scaled

# scale data (may want to scale up cluster centers)
def scale_up(data, data_max):
    data_scaled = data.copy()
    for i in range(len(data.columns)):
        col_i = data.columns[i]
        data_scaled[col_i] = data[col_i] * data_max[i]
        
    return data_scaled

In [ ]:
# number of records for sample
# could use a percentage of total records
# 1000 records good for basic development
nrecs = 10000

# create sample data
data_sample = select_sample(data[LWH], nrecs) 
# get maxes of sample for scaling data
data_max = data[LWH].max(axis=0)
# scale data to (0,1] for clustering
data_scaled = scale_down(data_sample, data_max)

# calculate volume for all sample records
data_sample['V'] = data_sample['L'] * data_sample['W'] * data_sample['H'] / 12**3

In [ ]:
# find clusters
min_clusters = 1
max_clusters = 96
step = 10

# kmeans can return: labels, centers, squared errors
# only labels (think: bin sizes) needed for this application
labels = kmeans_loop(data_scaled,
                     min_clusters,
                     max_clusters,
                     step)

binsets = pd.DataFrame(labels).T
bin_counts = list(range(min_clusters, max_clusters+1, step))
binsets.columns = ['set'+str(b).rjust(2, '0') for b in bin_counts]

In [ ]:
# assign binID for each set of bins to each record
data_all = pd.merge(data_sample, binsets.astype(str), left_index=True, right_index=True)
data_all.head()

In [ ]:
bin_cols = list(binsets.columns)
Vcols = ['V'+b for b in bin_cols]
bins = []
for b in bin_cols:
    bins_ = data_all.groupby(b)[LWH].max()
    bins_['V'] = bins_['L'] * bins_['W'] * bins_['H'] / 12**3
    bins.append(bins_)
    
# merge bin data with sample data
for i in range(len(bins)):
    data_all = data_all.merge(bins[i]['V'], on=bin_cols[i], suffixes=('',bin_cols[i]))
    
data_all.head()

In [ ]:
utilization = 1/data_all[Vcols].div(data_all.V, axis=0)
utilization.columns = utilization.columns.str.replace("Vset", "util", regex=True)

util_melt = pd.melt(utilization, var_name='u', value_name='utilization')
util_melt['bins'] = util_melt.u.str[-2:].astype(int)

# calculate quartiles for each bin set
utilization.quantile(q=[0.10, 0.25, 0.5, 0.75, 0.90], axis=0, numeric_only=True)

data_all = pd.merge(data_all, utilization, left_index=True, right_index=True)

In [ ]:
set21 = data_all.groupby('set21')['V'].count().sort_values(ascending=False)
set21.name = 'cases'
set21 = pd.merge(set21, bins[2], left_index=True, right_index=True)

fig = px.bar(set21,
             x='V', y='cases',
             color_discrete_sequence=['blue'])

fig.show()

In [ ]:
fig = px.scatter(data_all.sort_values(by='V'), 
                 x='H', y='set91', 
                 color_discrete_sequence=['blue'])

fig.show()

In [ ]:
fig = px.violin(util_melt, x='bins', y="utilization", box=True)
fig.update_yaxes(tickformat=".0%")
fig.add_hline(y=0.8, line_width=3, line_dash="dash", line_color="green")
fig.show()

In [ ]:
# calculate total utilization
util = 1 / (data_all[Vcols].sum() / sum(data_all.V))
util_summary = pd.DataFrame({'binIDs':util.index, 'util':util.values})
util_summary['bins'] = util_summary.binIDs.str[-2:].astype(int)
util_summary['util_delta'] = util_summary.util.diff()
util_summary['bins_delta'] = util_summary.bins.diff()
util_summary['util_improvement'] = util_summary.util_delta / util_summary.bins_delta

In [ ]:
fig = px.scatter(util_summary, 
                 x='bins', y='util', 
                 range_y=[0,1], 
                 color_discrete_sequence=['blue'])
fig.update_traces(marker={'size':10,
                          'color':'blue',
                          'symbol':'square'})

fig.update_yaxes(tickformat=".0%")
fig.add_hline(y=0.8, line_width=3, line_dash="dash", line_color="green")

fig.show()

In [ ]:
fig = px.scatter_3d(data_all, x='L', y='W', z='H', 
                    color='set11',
                    color_discrete_sequence=px.colors.qualitative.T10,
                    opacity = 0.25)

#fig.add_trace(
#    go.Scatter3d(
#        x = centers['L'],
#        y = centers['W'],
#        z = centers['H'],
#        mode = 'markers',
#        marker_symbol = 'cross',
#        marker_color = 'gray',
#        marker_size = 30)
#)

                    
fig.show()